In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
import os
import shutil
import pandas as pd

## Data Preprocessing

In [3]:
if not os.path.exists("./covid19dataset"):
    os.mkdir('./covid19dataset')
if not os.path.exists("./covid19dataset/train"):
    os.mkdir('./covid19dataset/train')
if not os.path.exists('./covid19dataset/val'):
    os.mkdir('./covid19dataset/val')

In [4]:
train_path = "./covid19dataset/train"
if not os.path.exists(train_path+'/Covid19'):
    os.mkdir(train_path+'/Covid19')
val_path = "./covid19dataset/val"
if not os.path.exists(val_path+'/Covid19'):
    os.mkdir(val_path+'/Covid19')

In [5]:
DataPath = "./covid-chestxray-dataset/images"
Metadata = "./covid-chestxray-dataset/metadata.csv"

In [6]:
meta = pd.read_csv(Metadata)
meta.head(n=5)

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,needed_supplemental_O2,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 27
0,2,0.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,infiltrate in the upper lobe of the left lung,NaN,NaN
1,2,3.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,progressive infiltrate and consolidation,NaN,NaN
2,2,5.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,progressive infiltrate and consolidation,NaN,NaN
3,2,6.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,progressive infiltrate and consolidation,NaN,NaN
4,4,0.0,F,52.0,COVID-19,NaN,NaN,NaN,NaN,NaN,...,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN


In [7]:
meta.columns

Index(['patientid', 'offset', 'sex', 'age', 'finding', 'survival', 'intubated',
       'intubation_present', 'went_icu', 'needed_supplemental_O2', 'extubated',
       'temperature', 'pO2_saturation', 'leukocyte_count', 'neutrophil_count',
       'lymphocyte_count', 'view', 'modality', 'date', 'location', 'folder',
       'filename', 'doi', 'url', 'license', 'clinical_notes', 'other_notes',
       'Unnamed: 27'],
      dtype='object')

In [8]:
meta["view"]

0      PA
1      PA
2      PA
3      PA
4      PA
       ..
355    AP
356    PA
357    PA
358    PA
359    AP
Name: view, Length: 360, dtype: object

In [9]:
cnt = 0
for idx,row in meta.iterrows():
    if  row['finding'] == "COVID-19" and row["view"] == 'PA':
        cnt += 1
print(cnt)

145


In [10]:
i = 0
SPLIT = 0.8
lim = int(SPLIT*cnt)

for idx,row in meta.iterrows():
    if  row['finding'] == "COVID-19" and row["view"] == 'PA':
        filename = row['filename']
        if i <= lim: 
            despath = os.path.join(train_path+"/Covid19",filename)
        else:
            despath = os.path.join(val_path+"/Covid19",filename)
        srcpath = os.path.join(DataPath,filename)    
        shutil.copy2(srcpath,despath)
        print('Moving the file %d'%i)
        i += 1

print(cnt)

Moving the file 0
Moving the file 1
Moving the file 2
Moving the file 3
Moving the file 4
Moving the file 5
Moving the file 6
Moving the file 7
Moving the file 8
Moving the file 9
Moving the file 10
Moving the file 11
Moving the file 12
Moving the file 13
Moving the file 14
Moving the file 15
Moving the file 16
Moving the file 17
Moving the file 18
Moving the file 19
Moving the file 20
Moving the file 21
Moving the file 22
Moving the file 23
Moving the file 24
Moving the file 25
Moving the file 26
Moving the file 27
Moving the file 28
Moving the file 29
Moving the file 30
Moving the file 31
Moving the file 32
Moving the file 33
Moving the file 34
Moving the file 35
Moving the file 36
Moving the file 37
Moving the file 38
Moving the file 39
Moving the file 40
Moving the file 41
Moving the file 42
Moving the file 43
Moving the file 44
Moving the file 45
Moving the file 46
Moving the file 47
Moving the file 48
Moving the file 49
Moving the file 50
Moving the file 51
Moving the file 52
Mov

In [11]:
normal_path = "./chest_xray/train/NORMAL"

In [12]:
normal_train_path = "./covid19dataset/train/Normal"
normal_val_path = "./covid19dataset/val/Normal"
os.mkdir(normal_train_path)
os.mkdir(normal_val_path)

In [13]:
import random
files = os.listdir(normal_path)
random.shuffle(files)

In [14]:
for j in range(cnt):
    src_path = os.path.join(normal_path,files[j])
    if j <= lim:
        dest_path = os.path.join(normal_train_path,files[j])
    else:
        dest_path = os.path.join(normal_val_path,files[j])
    shutil.copy2(src_path,dest_path)
    print("Moving the files %d"%j)

Moving the files 0
Moving the files 1
Moving the files 2
Moving the files 3
Moving the files 4
Moving the files 5
Moving the files 6
Moving the files 7
Moving the files 8
Moving the files 9
Moving the files 10
Moving the files 11
Moving the files 12
Moving the files 13
Moving the files 14
Moving the files 15
Moving the files 16
Moving the files 17
Moving the files 18
Moving the files 19
Moving the files 20
Moving the files 21
Moving the files 22
Moving the files 23
Moving the files 24
Moving the files 25
Moving the files 26
Moving the files 27
Moving the files 28
Moving the files 29
Moving the files 30
Moving the files 31
Moving the files 32
Moving the files 33
Moving the files 34
Moving the files 35
Moving the files 36
Moving the files 37
Moving the files 38
Moving the files 39
Moving the files 40
Moving the files 41
Moving the files 42
Moving the files 43
Moving the files 44
Moving the files 45
Moving the files 46
Moving the files 47
Moving the files 48
Moving the files 49
Moving the